**Навигация по уроку**

1. [Библиотеки визуализации данных в Python](https://colab.research.google.com/drive/1IbLhzlqiJhhPAErmdQ9wuIUdDaSRlNUF)
2. [Задача об акциях Tesla](https://colab.research.google.com/drive/1jukOs54u301WtyQS-SbRxWxDstTCjEBq)
3. Домашняя работа

Используя датасет о стоимости акций Сбербанка с 01.01.2013 года:

https://storage.yandexcloud.net/academy.ai/SBER.csv

визуализируйте индикатор "Полосы Боллинджера", проанализируйте график, и предложите вариант торговли акциями Сбербанка с помощью этого инструмента.

**Подсказка.**
Индикатор выглядит как полоса из трех линий:

* линия посередине — это простая скользящая средняя (SMA) с периодом `ma_size`, обычно около 20 дней;

* верхняя и нижняя линии (BB) — построены на основе SMА, но к нему добавлено стреднеквадратичное отклонение:

```
 SMA = data['close'].rolling(ma_size).mean()
 BB_UP = SMA + data['close'].rolling(ma_size).std() * bol_size
 BB_DOWN = SMA - data['close'].rolling(ma_size).std() * bol_size
   
```

где bol_size - ширина коридора, подбирается по графику. Выберите такое его значение, чтобы по графику можно было принимать торговые решения.

In [ ]:
# загружаем датасет
!wget https://storage.yandexcloud.net/academy.ai/SBER.csv

--2024-12-17 07:04:42--  https://storage.yandexcloud.net/academy.ai/SBER.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180264 (176K) [text/csv]
Saving to: ‘SBER.csv’

SBER.csv            100%[===================>] 176.04K   252KB/s    in 0.7s    

2024-12-17 07:04:43 (252 KB/s) - ‘SBER.csv’ saved [180264/180264]



In [ ]:
# подключаем нужные библоитеки
import os
import numpy as np
import pandas as pd

import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
# создаем датафрейм
df = pd.read_csv('./SBER.csv')
df.index = range(len(df))

In [ ]:
# смотрим инфо, видим что коллнка всего одна, с разделителем ;
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2718 entries, 0 to 2717
Data columns (total 1 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   DATE;OPEN;HIGH;LOW;CLOSE;VOL  2718 non-null   object
dtypes: object(1)
memory usage: 21.4+ KB


In [ ]:
df.head()

,DATE;OPEN;HIGH;LOW;CLOSE;VOL
0,20130108;96.5000000;98.5000000;96.1200000;98.3...
1,20130109;98.4100000;98.6500000;97.8100000;98.2...
2,20130110;98.3500000;98.5500000;97.9600000;98.4...
3,20130111;98.8000000;99.7600000;98.4800000;99.5...
4,20130114;99.7200000;101.2700000;99.5700000;100...


In [ ]:
# выделим имена колонок
df_names=df.columns[0].split(';')
df_names

['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']

In [ ]:
# преобразуем в удобный для обработки вид
# берем единственную колонку и разделим ее строки по ';'
# с разнесением сразу на отдельные колонки с помощью expand
df=df.iloc[:,0].str.split(';', expand=True)

# вернем имена колонкам, т.к. после разбиения они сбросились в [0, 1, 2, 3,...]
df.columns=df_names

In [ ]:
df.head(2)

,DATE,OPEN,HIGH,LOW,CLOSE,VOL
0,2013-01-08,96.5000000,98.5000000,96.1200000,98.3700000,92329970
1,2013-01-09,98.4100000,98.6500000,97.8100000,98.2300000,59776760


In [ ]:
# преобразуем колонку даты в формат даты
df['DATE']=pd.to_datetime(df['DATE'])
df.head(2)

,DATE,OPEN,HIGH,LOW,CLOSE,VOL
0,2013-01-08,96.5000000,98.5000000,96.1200000,98.3700000,92329970
1,2013-01-09,98.4100000,98.6500000,97.8100000,98.2300000,59776760


In [ ]:
# построим график акций на основе датафрейма
fig = go.Figure (data=go.Candlestick(x=df['DATE'],
                 open=df['OPEN'],
                 high=df['HIGH'],
                 low=df['LOW'],
                 close=df['CLOSE'],
                 name='price'))
fig.update_xaxes(rangeslider_visible=True, rangeselector=dict(
    buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.show()

In [ ]:
# добавим SMA на график
ma_size=20
df[f'SMA_{ma_size}']=df.CLOSE.rolling(ma_size).mean().shift()

In [ ]:
fig.add_trace(go.Scatter(x=df.DATE, y=df[f'SMA_{ma_size}'], marker_color='blue', opacity=0.5, name=(f'SMA_{ma_size}')))
fig.show()

In [ ]:
# добавим полосы Боллинджера
bol_size=4
fig.add_trace(go.Scatter(x=df.DATE, y=df[f'SMA_{ma_size}'] + df.CLOSE.rolling(ma_size).std()*bol_size, marker_color='blue', opacity=0.25, name='BB-UP'))
fig.add_trace(go.Scatter(x=df.DATE, y=df[f'SMA_{ma_size}'] - df.CLOSE.rolling(ma_size).std()*bol_size, marker_color='blue', opacity=0.25, name='BB-DOWN'))
fig.show()

# при выполнении ячейки несколько раз новая линия графика прочерчивается столько раз, сколько выполнена ячейка.
# каким методом можно удалить или очистить лишние trace графики?
# нашел несколько, такие как clf(), close() и еще пара, но они не сработали
# а как удалить определенный trace не нашел

# так же хотел сделать по аналогии с осью X, добавить масштабирование оси Y
# fig.update_yaxes(rangeslider_visible=True)
# но так же не получилось

Индикатор Боллинджера показывает отклонение среднего значения цен. Сигнал на покупку/продажу появляется, когда цена пересекает верхнюю или нижнюю границу канала, что показывает значительное отклонение от рынка. Когда таких пересечений нет, индикатор не дает сигналов.

Так же имеет свойство к сужению канала во время длительного колебания цены вокруг определенного значения, и расширяется при значительном отклонении цены от среднего значения.

# Итог

Создали и вывели временные графики SMA и BB для показателей цен акций. Добавили возможность регулировать по времени ширину окна отображения графика.